# Task 1 - Apriori Algorithm for Recommender System (33 points)

**Task Definition:** In this programming assignment, you are required to implement the Apriori algorithm based on the lecture slides and apply it to mine frequent itemsets for recommendation. You are required to implement the algorithm from scratch by using only native Python libraries and NumPy. For efficiency you will need to convert the items to ids and sort them. Implement the algorithm based on the lecture slides and make use of native Python packages (such as collections and itertools).

**Input:** The provided input file (`movies.txt`) based on the MovieLens dataset[1] contains the favourite movies of 1850 users. [1] Each line in the file corresponds to a user and represents a list of movies the user likes.

An example:

*Avengers: Infinity War - Part II;Jurassic World: Fallen Kingdom*

In the example above, the corresponding user likes the movies "Avengers: Infinity War - Part II" and "Jurassic World: Fallen Kingdom".

**Output:** You need to implement the Apriori algorithm and use it to mine frequent itemsets. Set the relative minimum support to 0.02 and run the algorithm on the 1850 list of movies. In other words, you need to extract all the itemsets that have an absolute support larger or equal to 37.

[1] Harper, F.M., Konstan, J.A.: The movielens datasets: History and context. ACM Trans. Interact. Intell. Syst. 5(4) (dec 2015). https://doi.org/10.1145/2827872, https://doi.org/10.1145/2827872

In [1]:
# TODO: uncomment the packages you used, please do not import additional non-native packages
# You may change the imports to the following format: from [package] import [class, method, etc.]

import collections
import itertools
import numpy as np

## 1.1 Loading the data and preprocessing (3 points)
**Task:** Solve the tasks explained in the TODOs and comments.

In [2]:
!pip install matplotlib scikit-learn notebook numpy pandas scipy seaborn
import pandas as pd

#requirement.txt

In [3]:
# TODO: read the data from the input file /data/movies.txt (1 points)
import pandas as pd

#read the movies.txt file
movie_r=pd.read_csv("./data/movies.txt", sep="\t", header=0, encoding="utf-8") 
print(movie_r.head(10))

  Incredibles 2;Avengers: Infinity War - Part I;Avengers: Infinity War - Part II;Deadpool 2
0  Annihilation;John Wick: Chapter 3 – Parabellum...                                       
1  Avengers: Infinity War - Part I;Bohemian Rhapsody                                       
2              Green Book;Incredibles 2;Isle of Dogs                                       
3  Bohemian Rhapsody;Captain Marvel;Avengers: Inf...                                       
4  Deadpool 2;Incredibles 2;Avengers: Infinity Wa...                                       
5  Green Book;Venom;Spider-Man: Into the Spider-V...                                       
6  Annihilation;Roma;Joker;Deadpool 2;Hereditary;...                                       
7       Avengers: Infinity War - Part I;Isle of Dogs                                       
8  Spider-Man: Into the Spider-Verse;Avengers: In...                                       
9  Deadpool 2;Incredibles 2;Avengers: Infinity Wa...                            

In [4]:
# TODO: determine the unique items and map the item to ids (1 points)
unique_items = []
id_to_item = unique_items

item_to_id = {}

def movies(file_path):
    return[line.strip().split(';') for line in open(file_path, 'r', encoding='utf-8')]

def mapping(data):
    items=sorted(set(item for sublist in data for item in sublist))
    df=pd.DataFrame(items, columns=['item'])
    df['id']=df.index + 1
    item_to_id=dict(zip(df['item'], df['id']))  
    id_to_item=dict(zip(df['id'], df['item'])) 
    return items, item_to_id, id_to_item

file='./data/movies.txt'
data=movies(file)
unique_items, item_to_id, id_to_item=mapping(data)

print("unique item:", unique_items)
print("mapping item to ID:", item_to_id)
print("mapping ID to item:", id_to_item)

unique item: ['A Quiet Place', 'A Star Is Born', 'Alita: Battle Angel', 'Annihilation', 'Ant-Man and the Wasp', 'Aquaman', 'Avengers: Infinity War - Part I', 'Avengers: Infinity War - Part II', 'Bad Times at the El Royale', 'Bird Box', 'BlacKkKlansman', 'Black Mirror: Bandersnatch', 'Bohemian Rhapsody', 'Captain Marvel', 'Crazy Rich Asians', 'Deadpool 2', 'Eighth Grade', 'Fantastic Beasts and Where to Find Them 2', 'First Man', 'Free Solo', 'Game Night', 'Glass', 'Green Book', 'Hereditary', 'Incredibles 2', 'Isle of Dogs', 'John Wick: Chapter 3 – Parabellum', 'Joker', 'Jurassic World: Fallen Kingdom', 'Love, Simon', 'Mission: Impossible - Fallout', "Ocean's 8", 'Once Upon a Time in Hollywood', 'Pacific Rim: Uprising', 'Ralph Breaks the Internet', 'Red Sparrow', 'Roma', 'Searching', 'Shazam!', 'Solo: A Star Wars Story', 'Sorry to Bother You', 'Spider-Man: Far from Home', 'Spider-Man: Into the Spider-Verse', 'The Ballad of Buster Scruggs', 'The Cloverfield Paradox', 'Tomb Raider', 'Upgra

In [5]:
# TODO: map the items of the records to ids and sort each record (1 points)
# In the following tasks use the mapped records to compute the frequent itemsets.

def map_and_sort_records(data, item_to_id):
    #Map the items of each record to ids
    mapped_data=[[item_to_id[item] for item in record] for record in data]
    #Sort each mappedd record by id
    sorted_data=[sorted(record) for record in mapped_data]
    return sorted_data


sorted_records=map_and_sort_records(data, item_to_id)
print("Sorted records:", sorted_records)

Sorted records: [[7, 8, 16, 25], [4, 7, 8, 16, 25, 27, 31, 43], [7, 13], [23, 25, 26], [7, 13, 14, 43], [7, 8, 14, 16, 25, 40, 43], [16, 23, 43, 49], [4, 7, 16, 21, 24, 28, 37, 41, 44], [7, 26], [7, 26, 38, 43, 47], [7, 16, 25, 41, 44], [1, 4, 6, 8, 12, 13, 16, 18, 21, 27, 31, 33, 37, 38, 42, 43, 48], [1, 5, 7, 16, 24, 25, 26, 29, 40, 49], [6, 7, 8, 13, 14, 20, 23, 40, 42, 43], [1, 4, 7, 16], [4, 12, 15, 21, 29], [4, 21, 47], [13, 23, 28, 33, 43], [7, 26], [8, 11, 13, 26, 43, 44], [2, 7, 8, 17, 22, 25, 26, 35, 40, 43], [7, 8], [1, 2, 4, 10, 23, 38], [25, 30], [13, 16, 23, 33], [1, 2, 7, 9, 13, 16, 19, 21, 25, 26, 31], [4, 5, 7, 8, 12, 14, 16, 21, 39, 42, 43], [7, 8, 16, 21, 28, 42, 43, 47, 49], [7, 16], [3, 4, 5, 7, 8, 10, 14, 22, 23, 24, 27, 28, 33, 42, 43], [5, 7, 8, 25, 27, 28, 43], [7, 16], [7, 16, 26], [2, 7, 13, 15, 16], [1, 4, 8, 11, 19, 21, 23, 25, 26, 37, 40, 42, 43, 44, 48, 50], [5, 29, 31, 49], [1, 4, 10, 16, 20, 22, 24, 29, 36, 45, 46, 47], [8, 43], [1, 4, 5, 7, 11, 13, 16,

## 1.2 Apriori algorithm (21 points)
### A) Prune the infrequent items (3 points)
**Task:** Solve the tasks explained in the TODOs and comments.

In [6]:
# TODO: calculate the support of length-1 itemsets (1 points)

def calculate_support(data):
    item_counts = {}
    #calculate the total transactions
    total_transactions = len(data)
    
    #Count the occurrences of each item
    for record in data:
        for item in record:
            if item in item_counts:
                item_counts[item]+= 1
            else:
                item_counts[item]=1
    
    #Calculate the support for each item
    support = {item: count / total_transactions for item, count in item_counts.items()}
    return support

support_length_1 = calculate_support(data) #Calculate the support of Length-1 itemsets
print("Support of length-1 itemsets:")
for item, support_value in support_length_1.items():
    print(f"{item}: {support_value:.4f}")

Support of length-1 itemsets:
Incredibles 2: 0.2995
Avengers: Infinity War - Part I: 0.6162
Avengers: Infinity War - Part II: 0.3416
Deadpool 2: 0.4519
Annihilation: 0.2151
John Wick: Chapter 3 – Parabellum: 0.0914
Mission: Impossible - Fallout: 0.1686
Spider-Man: Into the Spider-Verse: 0.3578
Bohemian Rhapsody: 0.2314
Green Book: 0.2022
Isle of Dogs: 0.2135
Captain Marvel: 0.1541
Solo: A Star Wars Story: 0.1659
Venom: 0.1243
Roma: 0.0984
Joker: 0.0978
Hereditary: 0.1497
The Ballad of Buster Scruggs: 0.1081
Sorry to Bother You: 0.0811
Game Night: 0.1643
Searching: 0.0778
Upgrade: 0.1232
Us: 0.0962
Fantastic Beasts and Where to Find Them 2: 0.1000
A Quiet Place: 0.2595
Once Upon a Time in Hollywood: 0.1049
Black Mirror: Bandersnatch: 0.0886
Spider-Man: Far from Home: 0.1351
Aquaman: 0.1151
Jurassic World: Fallen Kingdom: 0.1000
Ant-Man and the Wasp: 0.2141
Free Solo: 0.0795
Crazy Rich Asians: 0.0843
BlacKkKlansman: 0.1330
Ralph Breaks the Internet: 0.0719
Eighth Grade: 0.0654
Glass: 0.0

In [7]:
# Store all frequent itemsets (keys) with their support (value) in the dictionary below.
# TODO: add the length-1 frequent items with their supports to frequent_itemsets (2 points)

frequent_itemsets = {}

def calculate_support(data):
    item_counts={}
    total_transactions=len(data)
    
    for record in data:
        for item in record:
            if item in item_counts:
                item_counts[item]+=1
            else:
                item_counts[item]=1
    
    return item_counts, total_transactions

item_counts, total_transactions=calculate_support(data) #calculate support from the datase
min_relative_support=0.02 #minimum relative support (0.02)
min_absolute_support=37 #minimum absolute support (37)

for item, count in item_counts.items():
    if count>=min_absolute_support:
        support=count / total_transactions
        frequent_itemsets[frozenset([item])]=support

print(f"Frequent length-1 itemsets (min_support={min_relative_support:.2f}, min_absolute_support={min_absolute_support}):")
for itemset, support in frequent_itemsets.items():
    print(f"{list(itemset)}: absolute support={support * total_transactions:.0f}, relative support={support:.4f}")
print(f"\nTotal number of frequent length-1 itemsets: {len(frequent_itemsets)}")

Frequent length-1 itemsets (min_support=0.02, min_absolute_support=37):
['Incredibles 2']: absolute support=554, relative support=0.2995
['Avengers: Infinity War - Part I']: absolute support=1140, relative support=0.6162
['Avengers: Infinity War - Part II']: absolute support=632, relative support=0.3416
['Deadpool 2']: absolute support=836, relative support=0.4519
['Annihilation']: absolute support=398, relative support=0.2151
['John Wick: Chapter 3 – Parabellum']: absolute support=169, relative support=0.0914
['Mission: Impossible - Fallout']: absolute support=312, relative support=0.1686
['Spider-Man: Into the Spider-Verse']: absolute support=662, relative support=0.3578
['Bohemian Rhapsody']: absolute support=428, relative support=0.2314
['Green Book']: absolute support=374, relative support=0.2022
['Isle of Dogs']: absolute support=395, relative support=0.2135
['Captain Marvel']: absolute support=285, relative support=0.1541
['Solo: A Star Wars Story']: absolute support=307, relati

### B) Determine the frequent n itemsets (15 points)
**Task:** Solve the tasks explained in the TODOs and comments.


In [8]:
# TODO: implement the apriori_gen algorithm based on the lecture slides
def apriori_gen(itemsets):
    # TODO: generate candidates (4 points)
    candidates = []
    itemsets = sorted(itemsets)  # Sort the itemsets for consistency
    
    for i in range(len(itemsets)):
        for j in range(i + 1, len(itemsets)):
            # Join itemsets if they share all but the last item
            itemset_1 = itemsets[i]
            itemset_2 = itemsets[j]
            
            # TODO: join two itemsets if the first k-1 items are the same (2 points)
            if itemset_1[:-1] == itemset_2[:-1]:  # Compare all except the last item
                candidate = tuple(sorted(set(itemset_1) | set(itemset_2)))  #Merge into a new itemset
                candidates.append(candidate)
    # TODO: prune the candidates and return them (4 points)
    pruned_candidates = []
    for candidate in candidates:
        subsets = [candidate[:i] + candidate[i+1:] for i in range(len(candidate))]
        if all(subset in itemsets for subset in subsets):  # Only valid candidates
            pruned_candidates.append(candidate)
    return pruned_candidates

# reference : https://github.com/egbavou/APRIORI_ALGORITHME/blob/main/apriori.py

In [9]:
# TODO: implement an algorithm to calculate the support of the given itemset (2 points)
# You do not need to implement a Hash Tree for calculating the supports.

def calculate_support(itemset, data):
    itemset_count = 0 #Counter to keep track of how many transactions contain the itemset

    for transaction in data:
        #If the itemset is a subset of the current transaction, increment the count
        if set(itemset).issubset(set(transaction)):
            itemset_count += 1

    #Calculate the support as the fraction of transactions containing the itemset
    support = itemset_count / len(data)
    return support

#Example
#Assume you have an itemset (like (16: 'Deadpool 2', '43: 'Spider-Man: Into the Spider-Verse)) and the sorted records
itemset = (16,43)
support_value = calculate_support(itemset, sorted_records)
print(f"Support of itemset {itemset}: {support_value:.4f}")

Support of itemset (16, 43): 0.1822


In [11]:
import itertools

def find_frequent_1_itemsets(data, min_support):
    item_counts = {}
    total_transactions = len(data)
    
    for transaction in data:
        for item in transaction:
            if item in item_counts:
                item_counts[item] += 1
            else:
                item_counts[item] = 1
    
    # Find frequent 1-itemsets based on minimum support
    frequent_1_itemsets = {frozenset([item]): count 
                           for item, count in item_counts.items() 
                           if count >= min_support * total_transactions}
    
    return frequent_1_itemsets, total_transactions

def apriori_gen(L_k_minus_1):
    candidates = set()
    for i in range(len(L_k_minus_1)):
        for j in range(i + 1, len(L_k_minus_1)):
            itemset_1 = list(L_k_minus_1[i])
            itemset_2 = list(L_k_minus_1[j])
            itemset_1.sort()
            itemset_2.sort()
            
            if itemset_1[:-1] == itemset_2[:-1]:  # Compare all except the last item
                candidate = frozenset(itemset_1 + [itemset_2[-1]])
                if len(candidate) == len(itemset_1) + 1:
                    candidates.add(candidate)
    
    # Pruning step
    pruned_candidates = set()
    for candidate in candidates:
        is_valid = True
        for subset in itertools.combinations(candidate, len(candidate) - 1):
            if frozenset(subset) not in L_k_minus_1:
                is_valid = False
                break
        if is_valid:
            pruned_candidates.add(candidate)
    
    return list(pruned_candidates)

def movies(file_path):
    return [line.strip().split(';') for line in open(file_path, 'r', encoding='utf-8')]

# Load data
file = './data/movies.txt'
data = movies(file)

# Set minimum support
min_support = 0.02  # 2%
min_absolute_support = 37  # 0.02 * 1850 ≈ 37

# TODO: set the initial frequent itemsets which needs to be the input of the first iteration (1 point)
# (It will be updated at the end of each iteration.)
frequent_1_itemsets, total_transactions = find_frequent_1_itemsets(data, min_support)
frequent_n_itemsets = list(frequent_1_itemsets.keys())

# Dictionary to store all frequent itemsets
frequent_itemsets = {1: frequent_1_itemsets}

# TODO: set the correct loop condition until the Apriori algorithm should run (1 point)
k = 2
while frequent_n_itemsets:
    # Generate candidate itemsets
    candidates = apriori_gen(frequent_n_itemsets)
    
    # Calculate support for k-itemsets
    candidate_counts = {candidate: 0 for candidate in candidates}
    for transaction in data:
        for candidate in candidates:
            if candidate.issubset(set(transaction)):
                candidate_counts[candidate] += 1
    
    # TODO: filter out the frequent candidates (2 point)
    frequent_candidates = {itemset: count for itemset, count in candidate_counts.items() 
                           if count >= min_absolute_support}

    # TODO: add the frequent candidates to frequent_itemsets (1 point)
    if frequent_candidates:
        frequent_itemsets[k] = frequent_candidates

    # Replace the frequent_n_itemsets for the next iteration
    frequent_n_itemsets = list(frequent_candidates.keys())
    
    k += 1

# Print top 10 results for each k 
for k, itemsets in frequent_itemsets.items():
    print(f"Frequent {k}-itemsets (Top 10):")
    count = 0
    for itemset, support in itemsets.items():
        if count >= 10:  # Limit to 10 itemsets
            break
        print(f"{list(itemset)}: absolute support = {support}, relative support = {support/total_transactions:.4f}")
        count += 1
    print()


# Print whole results
for k, itemsets in frequent_itemsets.items():
    print(f"Frequent {k}-itemsets:")
    for itemset, support in itemsets.items():
        print(f"{list(itemset)}: absolute support = {support}, relative support = {support/total_transactions:.4f}")
    print()

print(f"Total number of frequent itemsets: {sum(len(itemsets) for itemsets in frequent_itemsets.values())}")

# reference1 : https://jk6722.tistory.com/entry/3-Apriori (This website is in Korean)
# reference2 : https://github.com/asaini/Apriori/blob/master/apriori.py

Frequent 1-itemsets (Top 10):
['Incredibles 2']: absolute support = 554, relative support = 0.2995
['Avengers: Infinity War - Part I']: absolute support = 1140, relative support = 0.6162
['Avengers: Infinity War - Part II']: absolute support = 632, relative support = 0.3416
['Deadpool 2']: absolute support = 836, relative support = 0.4519
['Annihilation']: absolute support = 398, relative support = 0.2151
['John Wick: Chapter 3 – Parabellum']: absolute support = 169, relative support = 0.0914
['Mission: Impossible - Fallout']: absolute support = 312, relative support = 0.1686
['Spider-Man: Into the Spider-Verse']: absolute support = 662, relative support = 0.3578
['Bohemian Rhapsody']: absolute support = 428, relative support = 0.2314
['Green Book']: absolute support = 374, relative support = 0.2022

Frequent 2-itemsets (Top 10):
['Mission: Impossible - Fallout', 'Fantastic Beasts and Where to Find Them 2']: absolute support = 60, relative support = 0.0324
['A Quiet Place', 'The Clover

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



### C) Save your results (3 points)

**Task:** Save all the frequent itemsets along with their absolute supports into a text file named `patterns.txt` and place it in the root of your zip file. Every line corresponds to exactly one frequent itemset and should be in the following format:

*support:movie1;movie2;movie3;...*

For example, suppose an itemset (Mission: Impossible - Fallout;A Star Is Born) has an absolute support 74, then the line corresponding to this frequent itemset in `patterns.txt` should be:

*74:Mission: Impossible - Fallout;A Star Is Born*

In [12]:
with open('patterns.txt', 'w', encoding='utf-8') as f:
    for k, itemsets in frequent_itemsets.items():
        for itemset, support in itemsets.items():
            f.write(f"{support}: {', '.join(itemset)}\n")

print("Save the results into a text file named 'patterns.txt'")

Save the results into a text file named 'patterns.txt'


## 1.3 Recommendation (9 points)

**Task:** Imagine you should recommend a movie to a user. You know that the user likes the movies “Ant-Man and the Wasp” and “Spider-Man: Far from Home”. Based on the result of the Apriori algorithm, give a movie recommendation for this user by maximizing the confidence that the user will like the movie. Explain your choice and report the confidence score for your recommendation.(6 points)

**Report:** Explain your method (comments in code or summary) and display your recommendations with the confidence scores. (3 points)


In [13]:
#Find itemsets from frequent itemsets that contain two specific movies
liked_movies = {'Ant-Man and the Wasp', 'Spider-Man: Far from Home'}

#Search for itemsets in frequent itemsets that include both movies
recommendations = []
for k, itemsets in frequent_itemsets.items():
    for itemset, support in itemsets.items():
        if liked_movies.issubset(itemset):
            # If an itemset contains both movies, recommend the other movies in the itemset
            remaining_movies = itemset - liked_movies
            if remaining_movies:
                recommendations.append((remaining_movies, support))

#Find the recommendation with the highest confidence score
best_recommendation = None
best_confidence = 0
for recommendation, support in recommendations:
    #Calculate confidence: Support(A ∪ B) / Support(A)
    total_support_liked = sum(support for k, itemsets in frequent_itemsets.items() for itemset, support in itemsets.items() if liked_movies.issubset(itemset))
    confidence = support / total_support_liked
    if confidence > best_confidence:
        best_confidence = confidence
        best_recommendation = recommendation

# Output the recommended movies and their confidence score
if best_recommendation:
    print(f"Recommended movie(s): {', '.join(best_recommendation)}")
    print(f"Confidence score: {best_confidence:.4f}")
else:
    print("No recommendations")

#reference : https://github.com/asaini/Apriori/blob/master/apriori.py

Recommended movie(s): Avengers: Infinity War - Part I
Confidence score: 0.0037
